In [12]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [13]:
import re

# ------- regex: match outer columns + [ ... ] payload -------
_LINE_RE = re.compile(r"^([^,]+),([^,]+),([^,]+),([^,]+),([^,]+),(\[.*\])\s*$")
_SEQ_RE  = re.compile(r"^\[(\d+),(.*)\]$")
_BRACE_RE = re.compile(r"\{([^}]*)\}")

# ------- enum maps (based on BEDOFIH / eurofidai guide) -------
ACTION_MAP = {
    1: "New",
    2: "Delete (by Previous Priority)",
    3: "Delete All (by side or whole book)",
    4: "Modify (no loss of priority)",
    5: "Retransmission (full book refresh)",
    6: "Modify (loss of priority)",
}

SIDE_MAP = {
    1: "Buy",
    2: "Sell",
}

# OrderType: keep common ones + fallback
ORDER_TYPE_MAP = {
    1: "Market",
    2: "Limit",
    3: "Stop",
    4: "Stop Limit",
    5: "Market If Touched",
    6: "Market to Limit",
    7: "Pegged",      # may vary by venue/version; keep as generic label
    8: "Iceberg",     # may vary
}

def decode_orderupdate(
    path: str,
    tz_local: str = "Europe/Paris",
    max_groups: int | None = None,
) -> pd.DataFrame:
    """
    Decode EUROFIDAI OrderUpdate CSV into a flat DataFrame:
      - 1 row per {...} group (one order update inside the message)
      - adds readable labels for action/type/side
      - converts event_time (ns since epoch UTC) to UTC and local time strings

    Args:
        path: path to OrderUpdate_*.csv
        tz_local: timezone string, default Europe/Paris
        max_groups: optional cap for debugging (stop after N groups)

    Returns:
        pandas DataFrame
    """
    rows = []
    n_groups = 0

    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.rstrip("\n")
            m = _LINE_RE.match(line)
            if not m:
                continue

            md_seq, msg_type, reb, emm, event_time, seq = m.groups()
            if msg_type != "OrderUpdate":
                continue

            md_seq = int(md_seq)
            reb = int(reb)
            emm = int(emm)
            event_time_ns = int(event_time)

            sm = _SEQ_RE.match(seq)
            if not sm:
                continue

            no_updates = int(sm.group(1))
            body = sm.group(2)

            groups = _BRACE_RE.findall(body)
            for g in groups:
                fields = g.split(",")
                # repeating group has 9 fields; pad if missing
                if len(fields) < 9:
                    fields += [""] * (9 - len(fields))
                fields = fields[:9]

                def to_int(x):
                    x = x.strip()
                    return None if x == "" else int(x)

                def to_float(x):
                    x = x.strip()
                    return None if x == "" else float(x)

                symbol_index = to_int(fields[0])
                action = to_int(fields[1])
                priority = to_int(fields[2])
                prev_priority = to_int(fields[3])
                order_type = to_int(fields[4])
                price = to_float(fields[5])
                side = to_int(fields[6])
                qty = to_float(fields[7])
                peg_offset = to_float(fields[8])

                rows.append({
                    # outer message fields
                    "md_seq": md_seq,
                    "rebroadcast": reb,
                    "emm": emm,
                    "event_time_ns": event_time_ns,
                    "no_updates": no_updates,

                    # repeating group fields
                    "symbol_index": symbol_index,
                    "action": action,
                    "priority": priority,
                    "prev_priority": prev_priority,
                    "order_type": order_type,
                    "price": price,
                    "side": side,
                    "qty": qty,
                    "peg_offset": peg_offset,
                })

                n_groups += 1
                if max_groups is not None and n_groups >= max_groups:
                    break

            if max_groups is not None and n_groups >= max_groups:
                break

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    # ---- add time columns ----
    dt_utc = pd.to_datetime(df["event_time_ns"].astype("int64"), unit="ns", utc=True)
    df["event_time_utc"] = dt_utc.astype(str)
    df["event_time_local"] = dt_utc.dt.tz_convert(tz_local).astype(str)

    # ---- add readable labels ----
    df["action_name"] = df["action"].map(ACTION_MAP).fillna(df["action"].apply(lambda x: f"Unknown({x})"))
    df["side_name"] = df["side"].map(SIDE_MAP).fillna("")
    df["order_type_name"] = df["order_type"].map(ORDER_TYPE_MAP).fillna(df["order_type"].apply(lambda x: "" if pd.isna(x) else f"Type({int(x)})"))

    # ---- reorder columns (nice for notebook) ----
    col_order = [
        "event_time_local", "event_time_utc", "event_time_ns",
        "md_seq", "rebroadcast", "emm", "no_updates",
        "symbol_index",
        "action", "action_name",
        "priority", "prev_priority",
        "side", "side_name",
        "order_type", "order_type_name",
        "price", "qty", "peg_offset",
    ]
    col_order = [c for c in col_order if c in df.columns]
    df = df[col_order]

    return df


In [ ]:
path = r"C:\Users\Wentao\Desktop\EA_recherche\euronextparis\EuronextParis\EuronextParis_20191001_FR0000120578\FR0000120578\OrderUpdate_20191001_FR0000120578.csv"

df_ou = decode_orderupdate(path, max_groups=None)  # 调试可以先限制数量
df_ou.head()
df_ou.describe()

,event_time_ns,md_seq,rebroadcast,emm,no_updates,symbol_index,action,priority,prev_priority,side,order_type,price,qty
count,3.140100e+05,3.140100e+05,314010.000000,314010.0,314010.000000,314010.0,314010.000000,1.655460e+05,1.607920e+05,314009.000000,314009.000000,165194.000000,314010.000000
mean,1.569931e+18,3.198411e+06,0.003551,1.0,1.242164,1110491.0,1.716522,1.569914e+18,1.569930e+18,1.535392,1.998519,84.479007,101.301373
std,1.023829e+13,1.736239e+06,0.059483,0.0,2.157195,0.0,1.047568,3.379281e+14,3.691907e+13,0.498747,0.062442,1.115979,887.345692
min,1.569892e+18,3.331000e+03,0.000000,1.0,1.000000,1110491.0,1.000000,1.557379e+18,1.557380e+18,1.000000,1.000000,44.010000,0.000000
25%,1.569921e+18,1.670048e+06,0.000000,1.0,1.000000,1110491.0,1.000000,1.569921e+18,1.569921e+18,1.000000,2.000000,84.150000,0.000000
50%,1.569934e+18,3.321458e+06,0.000000,1.0,1.000000,1110491.0,2.000000,1.569934e+18,1.569934e+18,2.000000,2.000000,84.600000,45.000000
75%,1.569940e+18,4.694842e+06,0.000000,1.0,1.000000,1110491.0,2.000000,1.569940e+18,1.569940e+18,2.000000,2.000000,84.820000,100.000000
max,1.569944e+18,6.085664e+06,1.000000,1.0,25.000000,1110491.0,6.000000,1.569944e+18,1.569944e+18,2.000000,6.000000,123.000000,100000.000000


In [21]:
df_ou["event_time_local"].max()

'2019-10-01 17:40:00.005826254+02:00'

In [15]:
def convert_ns_to_datetime(df, column_index=4, new_column_name='datetime'):
    """
    Converts a column of nanosecond timestamps in a DataFrame to a new datetime column.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_index (int): The index of the column containing nanosecond timestamps.
        new_column_name (str): The name for the new column with formatted datetimes.

    Returns:
        pd.DataFrame: The DataFrame with the new datetime column.
    """
    # Get the name of the column by its index
    timestamp_col = df.columns[column_index]
    
    # Convert the nanosecond timestamp column to datetime objects
    # We specify the timezone as UTC and then convert to a local timezone, e.g., 'Europe/Paris'
    df[new_column_name] = pd.to_datetime(df[timestamp_col], unit='ns', utc=True).dt.tz_convert('Europe/Paris')
    
    return df


orderupdate_converted = convert_ns_to_datetime(orderupdate, column_index=4)

In [16]:
orderupdate_converted.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,3331,OrderUpdate,0,1,1569891667220498345,[1,{1110491,3,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,0.1,}],datetime
3334,OrderUpdate,1,1,1569891667220498345,[25,{1110491,5,1.569512e+18,NaN,2,85.90,2,100,},{1110491,5.0,1.569425e+18,NaN,2.0,74.00,1.0,100.0,},{1110491,5.0,1.568981e+18,NaN,2.0,76.80,1.0,40.0,},{1110491,5.0,1.569389e+18,NaN,2.0,83.71,1.0,50.0,},{1110491,5.0,1.569318e+18,NaN,2.0,86.0,2.0,30.0,},{1110491,5.0,1.569389e+18,NaN,2.0,87.0,2.0,80.0,},{1110491,5.0,1.569321e+18,NaN,2.0,86.50,2.0,140.0,},{1110491,5.0,1.569304e+18,NaN,2.0,79.70,1.0,12.0,},{1110491,5.0,1.568620e+18,NaN,2.0,90.0,2.0,293.0,},{1110491,5.0,1.569825e+18,NaN,2.0,89.0,2.0,15.0,},{1110491,5.0,1.569561e+18,NaN,2.0,89.00,2.0,25.0,},{1110491,5.0,1.569218e+18,NaN,2.0,94.90,2.0,6.0,},{1110491,5.0,1.563205e+18,NaN,2.0,70.85,1.0,35.0,},{1110491,5.0,1.557405e+18,NaN,2.0,86.93,2.0,15.0,},{1110491,5.0,1.565246e+18,NaN,2.0,69.80,1.0,5.0,},{1110491,5.0,1.569573e+18,NaN,2.0,87.50,2.0,50.0,},{1110491,5.0,1.568216e+18,NaN,2.0,77.20,1.0,132.0,},{1110491,5.0,1.568705e+18,NaN,2.0,77.0,1.0,13.0,},{1110491,5.0,1.569233e+18,NaN,2.0,82.20,1.0,6.0,},{1110491,5.0,1.568721e+18,NaN,2.0,74.00,1.0,100.0,},{1110491,5.0,1.569239e+18,NaN,2.0,90.00,2.0,1111.0,},{1110491,5.0,1.569596e+18,NaN,2.0,83.8,1.0,50.0,},{1110491,5.0,1.569250e+18,NaN,2.0,98.00,2.0,184.0,},{1110491,5.0,1.569502e+18,NaN,2.0,90.00,2.0,24.0,},{1110491,5.0,1.557407e+18,NaN,2.0,88.50,2.0,1540.0,}],1970-01-01 01:00:00.000000024+01:00
3335,OrderUpdate,1,1,1569891667220498345,[25,{1110491,5,1.566401e+18,NaN,2,72.00,1,6,},{1110491,5.0,1.569821e+18,NaN,2.0,90.00,2.0,100.0,},{1110491,5.0,1.569821e+18,NaN,2.0,85.60,2.0,50.0,},{1110491,5.0,1.568110e+18,NaN,2.0,75.97,1.0,40.0,},{1110491,5.0,1.569313e+18,NaN,2.0,100.0,2.0,100.0,},{1110491,5.0,1.569246e+18,NaN,2.0,77.9,1.0,55.0,},{1110491,5.0,1.569821e+18,NaN,2.0,87.00,2.0,1442.0,},{1110491,5.0,1.569314e+18,NaN,2.0,88.78,2.0,40.0,},{1110491,5.0,1.566368e+18,NaN,2.0,72.1,1.0,6.0,},{1110491,5.0,1.568007e+18,NaN,2.0,73.5,1.0,13.0,},{1110491,5.0,1.569399e+18,NaN,2.0,87.00,2.0,276.0,},{1110491,5.0,1.568723e+18,NaN,2.0,78.00,1.0,5.0,},{1110491,5.0,1.569512e+18,NaN,2.0,86.00,2.0,60.0,},{1110491,5.0,1.567499e+18,NaN,2.0,92.80,2.0,5.0,},{1110491,5.0,1.569396e+18,NaN,2.0,87.80,2.0,3.0,},{1110491,5.0,1.568293e+18,NaN,2.0,72.00,1.0,50.0,},{1110491,5.0,1.569857e+18,NaN,2.0,83.75,1.0,25.0,},{1110491,5.0,1.567072e+18,NaN,2.0,76.5,1.0,1.0,},{1110491,5.0,1.569841e+18,NaN,2.0,88.00,2.0,6.0,},{1110491,5.0,1.569313e+18,NaN,2.0,74.00,1.0,40.0,},{1110491,5.0,1.569217e+18,NaN,2.0,88.00,2.0,50.0,},{1110491,5.0,1.568973e+18,NaN,2.0,76.0,1.0,197.0,},{1110491,5.0,1.569597e+18,NaN,2.0,85.50,2.0,40.0,},{1110491,5.0,1.565096e+18,NaN,2.0,71.30,1.0,7.0,},{1110491,5.0,1.560145e+18,NaN,2.0,89.00,2.0,48.0,}],1970-01-01 01:00:00.000000007+01:00
3336,OrderUpdate,1,1,1569891667220498345,[25,{1110491,5,1.569512e+18,NaN,2,86.50,2,115,},{1110491,5.0,1.568706e+18,NaN,2.0,77.50,1.0,12.0,},{1110491,5.0,1.568993e+18,NaN,2.0,87.99,2.0,48.0,},{1110491,5.0,1.568129e+18,NaN,2.0,75.50,1.0,30.0,},{1110491,5.0,1.569218e+18,NaN,2.0,86.0,2.0,369.0,},{1110491,5.0,1.569389e+18,NaN,2.0,87.9,2.0,7.0,},{1110491,5.0,1.569579e+18,NaN,2.0,90.00,2.0,575.0,},{1110491,5.0,1.569312e+18,NaN,2.0,86.80,2.0,28.0,},{1110491,5.0,1.569598e+18,NaN,2.0,89.0,2.0,25.0,},{1110491,5.0,1.569414e+18,NaN,2.0,88.0,2.0,1557.0,},{1110491,5.0,1.567488e+18,NaN,2.0,75.38,1.0,42.0,},{1110491,5.0,1.569339e+18,NaN,2.0,70.50,1.0,15.0,},{1110491,5.0,1.557380e+18,NaN,2.0,98.01,2.0,15.0,},{1110491,5.0,1.569822e+18,NaN,2.0,80.00,1.0,30.0,},{1110491,5.0,1.569339e+18,NaN,2.0,75.56,1.0,230.0,},{1110491,5.0,1.569304e+18,NaN,2.0,89.00,2.0,14.0,},{1110491,5.0,1.566193e+18,NaN,2.0,70.00,1.0,14.0,},{1110491,5.0,1.569314e+18,NaN,2.0,87.5,2.0,125.0,},{11104